# Análise de séries temporais

Neste exemplo iremos analisar o caso de um empresa de eventos.

Por meio da análise do passado de um evento, o que a empresa poderá aprender?

## Carregar pacotes

In [ ]:
library(tidyverse)
library(magrittr)
library(lubridate)
library(tseries)
library(forecast)

## Carregar dados

In [ ]:
dados_vendas <- read_csv( "/home/vm-data-science/dados/dados_vendas_evento.csv" )

In [ ]:
dados_vendas %>% head

## Análises

### Análise gráfica da série

In [ ]:
dados_vendas %>% 
    ggplot( aes( x = data_venda, y = qtd_inscricoes ) ) +
    geom_line()

### Verificar estacionariedade

In [ ]:
adf.test( x = dados_vendas$qtd_inscricoes, alternative = 'stationary' )

Verificamos que a série não é estacionária.

### Integração - I(d)

Podemos verificar o grau de integração para obter a estacionariedade.

In [ ]:
adf.test( x = diff(dados_vendas$qtd_inscricoes, 1), alternative = 'stationary' )

Com 1 diferenciação podemos obter a estacionariedade.

O novo gráfico com a série diferenciada.

In [ ]:
dados_vendas %>% 
    mutate( qtd_dif = c(0,diff(qtd_inscricoes)) ) %>% 
    ggplot( aes( x = data_venda, y = qtd_dif ) ) +
    geom_line()

### Parâmetros AR(p)

Para ter uma idéia de quantos parâmetros AR podem ser necessários verificamos o gráfico PACF.

In [ ]:
Pacf( x = diff(dados_vendas$qtd_inscricoes, 1), lag.max = 15 )

Observamos 3 lags fora da linha tracejada. Portanto, temos uma idéia que a parte AR terá ordem de até 4 (maior lag foi 4).

### Parâmetros MA(q)

Para ter uma idéia de quantos parâmetros AR podem ser necessários verificamos o gráfico ACF.

In [ ]:
Acf( x = diff(dados_vendas$qtd_inscricoes, 1), lag.max = 15 )

Observamos 3 lags fora da linha tracejada. Portanto, temos uma idéia que a parte MA terá ordem de até 10 (ultimo lag foi 10). 

Porém, durante o diagnóstico do modelo, perceberemos que não será necessária a ordem 10.

### Estimação do modelo ARIMA(p,d,q)

Observamos que a integração é de ordem d = 1, p até 4 e q até 10.

Iremos testar vários modelos.

### Amostras de treino e teste

Em séries temporais não podemos dividir aleatoriamente as amostras. Devemos selecionar um passado e buscar prever o futuro.

In [ ]:
dados_vendas %>% dim

In [ ]:
dados_vendas_train <- dados_vendas %>% slice(1:72)
dados_vendas_test <- dados_vendas %>% slice(73:77)

#### Modelo ARIMA(4, 1, 0)

In [ ]:
modelo_4_1_0 <- Arima( dados_vendas_train$qtd_inscricoes, order = c(4, 1, 0 ) )
modelo_4_1_0

Podemos diagnosticar a qualidade do modelo analisando o ACF e PACF dos resíduos do modelo.

In [ ]:
Acf( modelo_4_1_0$residuals, lag.max = 15 )

In [ ]:
Pacf( modelo_4_1_0$residuals, lag.max = 15 )

Por meio dos gráficos, observamos que não há algum lag fora das linhas tracejadas.

Este resultado nos mostra que o modelo não possui problema de autocorrelação e também pode não ser necessário utilizar os lags da parte MA.

### Avaliação do modelo

In [ ]:
dados_vendas_test %<>% 
  mutate( modelo_4_1_0_pred = forecast( modelo_4_1_0, h = 5)$mean )

dados_vendas_test

Podemos verificar a qualidade das previsões.

In [ ]:
accuracy( dados_vendas_test$modelo_4_1_0_pred,
          dados_vendas_test$qtd_inscricoes )

Podemos testar diversos outros modelos.

Pelo momento, somente iremos testar mais um. Iremos incluir a variável ticket.

In [ ]:
modelo_4_1_0_x <- Arima(dados_vendas_train$qtd_inscricoes, 
                        order = c(4, 1, 0 ),
                        xreg =  dados_vendas_train %>% select( ticket ) )
modelo_4_1_0_x

Será que melhor na amostra de teste?

In [ ]:
dados_vendas_test %<>% 
  mutate( modelo_4_1_0_x_pred = forecast( modelo_4_1_0_x, 
                                          xreg = dados_vendas_test %>% 
                                             select( ticket ), h = 5)$mean )

dados_vendas_test

In [ ]:
accuracy( dados_vendas_test$modelo_4_1_0_x_pred,
          dados_vendas_test$qtd_inscricoes )

Podemos ainda testar as variaveis: "número de dias até o evento" e criar uma variável que indica "período distante" e "período próximo". Quais outros testes você faria?